# Database Examples 

Examples for the new database schema and how to use it.

In [1]:
! pip install psycopg2-binary python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# jupyter notebook auto reload
%load_ext autoreload
%autoreload 2

## Setup

Utility functions for setting up the database.

In [2]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def create_conn():
    conn = psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
    )
    return conn

In [6]:
def execute_query(query):
    conn = create_conn()
    with conn:
        with conn.cursor() as cur:
            cur.execute(query)
            result = cur.fetchall()
    return result

def execute_write_query(query):
    conn = create_conn()
    with conn:
        with conn.cursor() as cur:
            cur.execute(query)
        conn.commit()

## Tables

Creating the necessary database tables

In [5]:
# postgres table definitions

_table_media = """
    CREATE TABLE IF NOT EXISTS media (
        media_id SERIAL PRIMARY KEY,
        media_path VARCHAR(500) NOT NULL,
        original_path VARCHAR(500) NOT NULL,
        created_at TIMESTAMP DEFAULT NOW(),
        media_type VARCHAR(50) NOT NULL,
        sub_type VARCHAR(50) NOT NULL,
        size INTEGER NOT NULL,
        archive_name VARCHAR (50) NOT NULL,
        archive_id VARCHAR (50) NOT NULL,
        metadata JSONB NOT NULL
    );
"""

_table_features = """
    CREATE TABLE IF NOT EXISTS features (
        feature_id SERIAL PRIMARY KEY,
        feature_type VARCHAR(50) NOT NULL,
        version VARCHAR(20) NOT NULL,
        created_at TIMESTAMP DEFAULT NOW(),
        model_name VARCHAR(200) NOT NULL,
        model_params JSONB NOT NULL,
        data JSONB NOT NULL,

        embedding_size INTEGER,
        embedding_1024 vector (1024),
        embedding_2048 vector (2048),

        media_id INTEGER,

        CONSTRAINT FK_features_media_id FOREIGN KEY (media_id) 
            REFERENCES media (media_id)
    );
"""

execute_write_query(_table_media)
execute_write_query(_table_features)

## Fill tables with sample data

In [7]:
# create sample data for the tables, each media element can have multiple features

swiss_cities = ["Zurich", "Geneva", "Basel", "Lausanne", "Bern", "Winterthur", "Lucerne", "St. Gallen", "Lugano", "Biel/Bienne"]

for i in range(1, 21):
    _table_media_sample = f"""
        INSERT INTO media (media_path, original_path, media_type, sub_type, size, archive_name, archive_id, metadata)
        VALUES 
            ('/path/to/media{i}', '/path/to/original{i}', 'image', 'jpg', 500, 'archive{i}', 'archive_id{i}', '{{"key{i}": "value{i}", "city": "{swiss_cities[(i-1) % len(swiss_cities)]}"}}')
        RETURNING media_id;
    """
    execute_write_query(_table_media_sample)


In [8]:
_query = """SELECT * FROM media LIMIT 2;"""
execute_query(_query)

[(1,
  '/path/to/media1',
  '/path/to/original1',
  datetime.datetime(2023, 4, 26, 12, 54, 15, 684379),
  'image',
  'jpg',
  500,
  'archive1',
  'archive_id1',
  {'city': 'Zurich', 'key1': 'value1'}),
 (2,
  '/path/to/media2',
  '/path/to/original2',
  datetime.datetime(2023, 4, 26, 12, 54, 15, 730683),
  'image',
  'jpg',
  500,
  'archive2',
  'archive_id2',
  {'city': 'Geneva', 'key2': 'value2'})]

In [9]:
# insert some samples with vectors
import numpy as np

for i in range(1, 11):
    vector_1024 = np.random.rand(1024).tolist()
    vector_2048 = np.random.rand(2048).tolist()

    _table_features_sample_vectors = f"""
        INSERT INTO features (feature_type, version, model_name, model_params, data, media_id, embedding_size, embedding_1024)
        VALUES
            ('image', 'v1', 'resnet50', '{{"param1": "{i}"}}', '{{"data1": "{i}"}}', {i}, 1024, ARRAY[{','.join([str(x) for x in vector_1024])}])
        RETURNING feature_id;
    """
    execute_write_query(_table_features_sample_vectors)

    # At the moment we are only creating size 1024 vectors, for the sake of the next example queries to work, there can be only a single vector set per feature
    # _table_features_sample_vectors = f"""
    #     INSERT INTO features (feature_type, version, model_name, model_params, data, media_id, embedding_size, embedding_2048)
    #     VALUES
    #         ('image', 'v1', 'resnet50', '{{"param1": "{i}"}}', '{{"data1": "{i}"}}', {i}, 2048, ARRAY[{','.join([str(x) for x in vector_2048])}])
    #     RETURNING feature_id;
    # """

    # execute_write_query(_table_features_sample_vectors)


## Queries

### Scenario 1

We have a media object and we want to find the 5 most similar media objects.

In [18]:
_query = """
    WITH target_embedding AS (
    SELECT
        media_id,
        CASE
            WHEN embedding_size = 1024 THEN embedding_1024
            WHEN embedding_size = 2048 THEN embedding_2048
            ELSE NULL
        END AS embedding_vector
    FROM 
        features
    WHERE 
        media_id = 5
    )

    SELECT
    f.media_id,
    (target.embedding_vector <-> 
        CASE
        WHEN f.embedding_size = 1024 THEN f.embedding_1024
        WHEN f.embedding_size = 2048 THEN f.embedding_2048
        ELSE NULL
        END
    ) AS distance
    FROM
    features f,
    target_embedding target
    WHERE
    f.media_id != target.media_id
    ORDER BY
    distance ASC
    LIMIT 5;
"""

execute_query(_query)

[(10, 12.6901558729218),
 (2, 12.7666318065067),
 (7, 12.8138656753955),
 (9, 12.8490967504447),
 (1, 13.0124700409029)]

### Scenario 2

Find all media objects for Zurich.

In [10]:
# get all media objects that have city: "Zurich" (queried from the jsonb metadata field)
_query = """
    SELECT * FROM media WHERE metadata->>'city' = 'Zurich';
"""
execute_query(_query)

[(1,
  '/path/to/media1',
  '/path/to/original1',
  datetime.datetime(2023, 4, 26, 12, 54, 15, 684379),
  'image',
  'jpg',
  500,
  'archive1',
  'archive_id1',
  {'city': 'Zurich', 'key1': 'value1'}),
 (11,
  '/path/to/media11',
  '/path/to/original11',
  datetime.datetime(2023, 4, 26, 12, 54, 16, 171085),
  'image',
  'jpg',
  500,
  'archive11',
  'archive_id11',
  {'city': 'Zurich', 'key11': 'value11'})]

### Scenario 3

Get all images from Geneva

In [12]:
_query = """
    SELECT * FROM media WHERE metadata->>'city' = 'Geneva' AND media_type = 'image';
"""
execute_query(_query)


[(2,
  '/path/to/media2',
  '/path/to/original2',
  datetime.datetime(2023, 4, 26, 12, 54, 15, 730683),
  'image',
  'jpg',
  500,
  'archive2',
  'archive_id2',
  {'city': 'Geneva', 'key2': 'value2'}),
 (12,
  '/path/to/media12',
  '/path/to/original12',
  datetime.datetime(2023, 4, 26, 12, 54, 16, 210144),
  'image',
  'jpg',
  500,
  'archive12',
  'archive_id12',
  {'city': 'Geneva', 'key12': 'value12'})]

### Scenario 4

Fulltext string matching on jsonb fields

In [19]:
# query for cities that start with b
_query = """
    SELECT * FROM media WHERE metadata->>'city' LIKE 'B%';
"""
print([x[9]['city'] for x in execute_query(_query)])

_query = """
    SELECT * FROM media WHERE metadata->>'city' LIKE '%Gall%';
"""
[x[9]['city'] for x in execute_query(_query)]



['Basel', 'Bern', 'Biel/Bienne', 'Basel', 'Bern', 'Biel/Bienne']


['St. Gallen', 'St. Gallen']